# Antes de ejecutar este código es importante leer el archivo README de la carpeta.

# Limpieza de datos

## Primero el dataset de logs_entrenamiento:

In [ ]:
import pandas as pd

logs_entrenamiento = pd.read_csv('logs_entrenamiento.csv')

# Configuración de pandas
pd.set_option('display.max_columns', None) # Que se muestren todas las columnas

Se puede notar que los datos son de los momentos en que una persona interactúa con la página. Existen tipos de interacciones o eventos. Ellos estan asociados a una persona, una momento en el tiempo y una sección (chapter) y subsección (sequential) de página.

# Verificamos existencia de NA's
logs_entrenamiento.isna().sum()

Se observa que los NA estan asociados a las variables de chapter y sequential.

In [ ]:
logs_entrenamiento.info()

In [ ]:
# Se busca los grouped_event_type asociados a chapter

valores_unicos = logs_entrenamiento.loc[logs_entrenamiento['chapter'].isna(), 'grouped_event_type'].unique()
print(valores_unicos)

In [ ]:
# Se busca los grouped_event_type asociados a sequential

valores_unicos = logs_entrenamiento.loc[logs_entrenamiento['sequential'].isna(), 'grouped_event_type'].unique()
print(valores_unicos)

Se puede ver que los NA estan asociados a un típico específico de dato, es decir, es probable que estos NA no se deban a una falta de un dato existente, sino que, el tipo de registro asociado a edx no posee chapter ni sequential. En consecuencia, se decide no eliminar los NA

In [ ]:
import matplotlib.pyplot as plt

plt.hist(logs_entrenamiento['username'], bins=500, edgecolor='black')  # 'bins' define el número de barras en el histograma

# Personaliza el título y las etiquetas de los ejes
plt.title('Cantidad de veces que un usuario interactuó')

# Muestra el histograma
plt.show()

En la eliminación de outlayers, se debe notar que es posible que aquellas personas que menos interactúan con la página, sean las personas que tienen las peores notas. Por lo tanto, se decide eliminar a los outlayers superiores únicamente.

In [ ]:
# Calcula el número mínimo y máximo de registros basados en el 5% más extremo
umbral_minimo = logs_entrenamiento['username'].value_counts().quantile(0.00)
umbral_maximo = logs_entrenamiento['username'].value_counts().quantile(0.90)

# Filtra el DataFrame para eliminar las filas correspondientes
logs_entrenamiento_filtrado = logs_entrenamiento[logs_entrenamiento.groupby('username')['username'].transform('count').between(umbral_minimo, umbral_maximo)]

# Se omite la exportación del csv para este notebook en particular
#logs_entrenamiento_filtrado.to_csv('logs_entrenamiento_filtrado.csv', index=True)

## Ahora el dataset de notas:

In [ ]:
notas = pd.read_csv('notas.csv')

In [ ]:
# Se modifica Not Atempted por -1
notas = notas.replace('Not Attempted', -1)

for col in notas.columns:
    columnas_listas = ['Unnamed: 0','Username','Grade','Grade Scaled','Quiz (Avg)']
    if col in columnas_listas:
        continue
    
    notas[col] = pd.to_numeric(notas[col], errors='coerce')

In [ ]:
# Se verifica que funcionó
notas.info()

In [ ]:
# Verificamos existencia de NA's (No hay, se traspasó bien el type object a float)
notas.isna().sum()

In [ ]:
# De la revisión del CSV, se observa que Quiz 17 contiene sólo casos 'No Atempted'
notas['Quiz 17: Introducción'].describe()

In [ ]:
# Eliminamos dicha columna
notas = notas.drop('Quiz 17: Introducción', axis=1)    

# Se observa que las columnas Grade y Grade Scalated contienen la misma información, se decide por querdarse con la original
notas = notas.drop('Grade Scaled',axis=1)

### Tratamiento de outliers

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

notas_df = notas

# Reemplaza 'Not Attempted' con -1 y convierte las cadenas a flotantes
quiz_columns = [col for col in notas_df.columns if 'Quiz' in col]
notas_df[quiz_columns] = notas_df[quiz_columns].replace('Not Attempted', -1).astype(float)

# Determinamos el número de filas para los subplots
n_rows = (len(quiz_columns) + 1) // 2

# Crear una figura con subplots en dos columnas
fig = make_subplots(rows=n_rows, cols=2, subplot_titles=quiz_columns)

# Añadir un histograma para cada cuestionario en la posición adecuada
for idx, col in enumerate(quiz_columns):
    fig.add_trace(
        go.Histogram(x=notas_df[col], name=col),
        row=(idx // 2) + 1,  # Incrementar la fila después de cada dos cuestionarios
        col=(idx % 2) + 1    # Alternar entre columna 1 y 2
    )

# Actualizar el layout para que se ajuste bien
fig.update_layout(
    height=300 * n_rows,  # Ajustar la altura si es necesario para acomodar todos los subplots
    title_text="Distribución de Puntuaciones por Cuestionario",
    showlegend=False
)

# Mostrar la figura
fig.show()


Al analizar los datos se puede ver que las variables se encuentran distribuidas dentro de rangos esperados, es decir, dentro de (0,1) con valores ocasionales de -1, los cuales son los eventos en que un estudiante decidió no intentar el Quiz.

En el caso excepcional del Quiz 1, se observa una alta tasa de No Atempt, esto se puede deber a que el curso está comenzando y que el primer quiz es sin nota y ademas introductorio a EOL.

Otra cosa a tomar en cuenta, es la fuerte concentración de registros alrededor de la nota 1.0 en todos los EOL, esto puede significar un trabajo futuro para el balanceo de clases.


In [ ]:
import seaborn as sns

sns.boxplot(x=notas['Grade'])
plt.title('Box Plot de notas')
plt.show()

Se observa que en el agregado de notas, se tiene una fuerte tendencia a centrarse entre el 0.8 y el 1.0, se decide no eliminar aquellos outliers que nunca respondieron ningún quiz, por lo que obtuvieron nota 0. Esto porque pueden contener información de si la persona pertenece a la clase a identificar.

Hay 26 outliers, lo que representa aproximadamente el 4.43% del total de datos.
Los límites para considerar un valor como atípico están dados por:
Límite inferior : 0.655
Límite superior : 1.175
Esto significa que cualquier calificación por debajo de 0.655 se considera un outlier. Dado que la escala de calificaciones no ha sido especificada, asumimos que está entre 0 y 1, donde 1 es la calificación más alta posible.

Las estadísticas descriptivas de los outliers son:

Media (Mean): 0.41, lo que indica que en promedio, los outliers están por debajo del promedio general.
Desviación estándar (Std): 0.25, lo que muestra una variabilidad considerable entre los valores atípicos.
Mínimo (Min): 0.00, la calificación más baja posible, lo que sugiere que algunos estudiantes no obtuvieron ningún punto.
Cuartiles (25%, 50%, 75%): Los valores de los cuartiles indican que la mitad de los outliers tiene calificaciones entre 0.2475 y 0.5975, lo que muestra que no todos los outliers están en el extremo más bajo.
Este pequeño porcentaje de outliers podría representar casos de estudiantes que tuvieron un rendimiento significativamente por debajo del promedio.
A partir de esto consideramos importante no eliminar estos outliers dado que representan personas con baja califacion, por lo cual puede ser importante en el modelo que estas personas sean parte de los datos, para que el modelo tambien aprenda de estos usuarios, que pueden ser parte importante de los que reprobaron el ramo.

A partir de los outliers encontrados en logs entrenamiento, los quitamos del dataframe de notas

In [ ]:
# Obtenemos los usuarios del df filtrados
usernames_filtrados = set(logs_entrenamiento_filtrado['username'].unique())
len(usernames_filtrados)

In [ ]:
# Obtenemos los usuarios del df sin filtrar
logs_entrenamiento = pd.read_csv('logs_entrenamiento.csv')
usernames_sin_filtrar = set(logs_entrenamiento['username'].unique())
len(usernames_sin_filtrar)

In [ ]:
usernames_diferencia = usernames_sin_filtrar - usernames_filtrados
# Obtenemos los usuarios que fueron eliminados, es decir, los outliers del logs entrenamiento
len(usernames_diferencia)

In [ ]:
notas_df_filtrado = notas_df[~notas_df['Username'].isin(usernames_diferencia)]
len(notas_df_filtrado)

# Cruce de los datos y generación de variables

In [ ]:
eventos = logs_entrenamiento_filtrado.copy()

notas = notas_df_filtrado.copy()

chapter = pd.read_csv('/map_displayname_to_chap_seq_id.csv')

column_names_df = notas.columns


column_names_eventos = eventos.columns

# Print the column names
print("Column names of 'df':", column_names_df)
print("Column names of 'eventos':", column_names_eventos)

# Rename a single column
notas.rename(columns={'Username': 'username'}, inplace=True)

notas = notas.replace('Not Attempted', 0)

notas.head()